In [1]:
from util import setup_notebook
setup_notebook()
%load_ext autoreload
%autoreload 2

## Plan
* Für jedes Dataset gibt es eine offenders.csv
* Kombiniere mit Original-CSV von untreated und (bester?) Matrix-Strategie
* Für jeden Build aus den offenders, berechne APFD von beiden und vergleiche.

In [7]:
from testmining import apfd
import pandas as pd

offenders = pd.read_csv("../output/square@okhttp/square@okhttp-pr.csv")
recently_failed = apfd.read_tests("../output/square@okhttp/baseline/okhttp@recently-failed.csv")
matrix = apfd.read_tests("../output/square@okhttp/baseline/okhttp@matrix-file-similarity.csv")

In [8]:
def go(df, job):
    entries = df[df['travisJobId'] == job]
    return apfd.apfd(apfd.apfd_plot(entries))

df = pd.DataFrame({
    'tr_job_id': offenders['tr_job_id'],
    'recently_failed': offenders['tr_job_id'].apply(lambda x: go(recently_failed, x)),
    'matrix': offenders['tr_job_id'].apply(lambda x: go(matrix, x)),
})

df['diff'] = df['matrix'] - df['recently_failed']
df.head()

,tr_job_id,recently_failed,matrix,diff
0,3077822,0.557759,0.557759,0.000000
1,3103902,0.437931,0.655172,0.217241
2,3144121,0.456667,0.666667,0.210000
3,3153783,0.671053,0.671053,0.000000
4,3168438,0.529412,0.575980,0.046569


In [9]:
# How many observations?
len(df)

186

In [10]:
# How many times was matrix better than recently failed?
(df['diff'] > 0).sum()

33

In [11]:
# Which strategy scored more often?
df['diff'].sum()

-4.748594906397005